In [6]:
#from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, Seq2SeqTrainingArguments, Seq2SeqTrainer
from datasets import load_dataset

# Load your dataset
dataset = load_dataset('cnn_dailymail', '3.0.0')

device = "mps" if torch.backends.mps.is_available () else "cpu"
model.to(device)

# Convert the dataset to a Pandas DataFrame
df = dataset['train'].to_pandas()

print(df.head())

                                             article  \
0  LONDON, England (Reuters) -- Harry Potter star...   
1  Editor's note: In our Behind the Scenes series...   
2  MINNEAPOLIS, Minnesota (CNN) -- Drivers who we...   
3  WASHINGTON (CNN) -- Doctors removed five small...   
4  (CNN)  -- The National Football League has ind...   

                                          highlights  \
0  Harry Potter star Daniel Radcliffe gets £20M f...   
1  Mentally ill inmates in Miami are housed on th...   
2  NEW: "I thought I was going to die," driver sa...   
3  Five small polyps found during procedure; "non...   
4  NEW: NFL chief, Atlanta Falcons owner critical...   

                                         id  
0  42c027e4ff9730fbb3de84c1af0d2c506e41c3e4  
1  ee8871b15c50d0db17b0179a6d2beab35065f1e9  
2  06352019a19ae31e527f37f7571c6dd7f0c5da37  
3  24521a2abb2e1f5e34e6824e0f9e56904a2b0e88  
4  7fe70cc8b12fab2d0a258fababf7d9c6b5e1262a  


In [1]:
from transformers import AutoModelForSeq2SeqLM
from transformers import AutoTokenizer
from transformers import Seq2SeqTrainingArguments 
from transformers import Seq2SeqTrainer
from datasets import load_dataset
import torch

# Load your dataset
dataset = load_dataset('cnn_dailymail', '3.0.0')

# Load a pre-trained model and tokenizer suitable for sequence-to-sequence tasks
model_name = "facebook/bart-large-cnn"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

device = "mps" if torch.backends.mps.is_available () else "cpu"
model.to(device)

# Define a function to preprocess your data
def preprocess_data(example):
    input_text = example["article"]
    target_text = example["highlights"]
    
    # Tokenize the input and target text using the __call__ method
    input_tokens = tokenizer(input_text, target_text, max_length=512, truncation=True, padding='max_length', return_tensors="pt", add_special_tokens=True)
    
    return {
        "input_ids": input_tokens.input_ids[0],
        "attention_mask": input_tokens.attention_mask[0],
        "labels": input_tokens.input_ids[0],  # Use input_ids for labels (same as input)
    }


# Apply preprocessing to your dataset
tokenized_dataset = dataset.map(preprocess_data, batched=False)

# Define the training arguments
training_args = Seq2SeqTrainingArguments(
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    output_dir="./bert-summarizer",
    save_total_limit=1,
    evaluation_strategy="steps",
    eval_steps=1000,
    num_train_epochs=1,
    save_steps=1000,
    logging_steps=1000,
    push_to_hub=False,
    report_to="tensorboard",
)

# Initialize the trainer
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["validation"],
    tokenizer=tokenizer,
)

# Fine-tune the model
trainer.train()

# Save the fine-tuned model
trainer.save_model("./bert-summarizer-fine-tuned")

# You can now use this fine-tuned model for text summarization tasks

/Users/isab7070/anaconda3/envs/mynlp/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
  0%|          | 9/143557 [22:47<7830:06:48, 196.37s/it]